<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-and-setup" data-toc-modified-id="Imports-and-setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports and setup</a></span></li><li><span><a href="#Dataframes" data-toc-modified-id="Dataframes-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dataframes</a></span><ul class="toc-item"><li><span><a href="#Create-pandas-dataframes-with-attributes:-filepath,-gal,-rc,-gau" data-toc-modified-id="Create-pandas-dataframes-with-attributes:-filepath,-gal,-rc,-gau-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Create pandas dataframes with attributes: filepath, gal, rc, gau</a></span><ul class="toc-item"><li><span><a href="#simulated-dataset" data-toc-modified-id="simulated-dataset-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>simulated dataset</a></span></li></ul></li><li><span><a href="#split-dataframe-to-two:-one-will-serve-as-test-dataset" data-toc-modified-id="split-dataframe-to-two:-one-will-serve-as-test-dataset-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>split dataframe to two: one will serve as test dataset</a></span></li><li><span><a href="#inspect-dataframes" data-toc-modified-id="inspect-dataframes-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>inspect dataframes</a></span></li></ul></li><li><span><a href="#Set-parameters-dictionary" data-toc-modified-id="Set-parameters-dictionary-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Set parameters dictionary</a></span></li><li><span><a href="#DataGenerators:-use-flow_from_dataframe-class" data-toc-modified-id="DataGenerators:-use-flow_from_dataframe-class-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>DataGenerators: use flow_from_dataframe class</a></span></li><li><span><a href="#CNN-Model" data-toc-modified-id="CNN-Model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>CNN Model</a></span></li><li><span><a href="#Define-callbacks" data-toc-modified-id="Define-callbacks-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Define callbacks</a></span></li><li><span><a href="#Model-fit" data-toc-modified-id="Model-fit-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Model fit</a></span></li><li><span><a href="#Plot-learning-curves" data-toc-modified-id="Plot-learning-curves-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Plot learning curves</a></span></li><li><span><a href="#Evaluation-on-test-dataset" data-toc-modified-id="Evaluation-on-test-dataset-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Evaluation on test dataset</a></span></li><li><span><a href="#Predict-on-single-images" data-toc-modified-id="Predict-on-single-images-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Predict on single images</a></span></li><li><span><a href="#General-sandbox-area" data-toc-modified-id="General-sandbox-area-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>General sandbox area</a></span></li></ul></div>


CNN with regression and double output: rc and gal rotation (BOA beamline naming convention), or else y-axis and x-axis rotation, respectively (McStas naming convention).




### Imports and setup

In [ ]:
import numpy as np 
import pandas as pd 
import os
import glob
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import cv2
from time import time
import seaborn as sns

from tensorflow.keras import models, layers,optimizers, metrics, backend, losses
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Conv2D, MaxPool2D, GlobalMaxPool2D, Dropout,Flatten
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices())

### Dataframes  

Get the rotation values from the name of the files and create pandas dataframes with file paths and attributes.

#### Create pandas dataframes with attributes: filepath, gal, rc, gau

In [ ]:
def get_rot_attributes(filepath):
    """ Reads from name of image file the values of the three angles and unique file's ID
        Keyword Arguments: 
        filepath -- the full path of the image file
    """
    try:
        path,file=os.path.split(filepath)
        file,ext=os.path.splitext(file)
        _,params = file.split('ellipse_rot_')
        gal,rc,gau,ID_t6 = params.split('_')

        return gal,rc,ID_t6
    except Exception as e:
        return None,None,None
        

##### simulated dataset

In [ ]:
# Set path of directory where image files should be read from and create list of attributes from all files.

data_dir_sim='full_path_location_of_images_directory' # choose directory where all images to be used are stored
filepaths_sim=glob.glob(os.path.join(data_dir_sim,'*.jpg')) # if not jpg format, change extension to match. Make sure TF supports fileformat.
random.shuffle(filepaths_sim)
attributes_sim=list(map(get_rot_attributes,filepaths_sim))

In [ ]:
# Create one dataframe with above attributes

dfsim=pd.DataFrame(attributes_sim)
dfsim['file']=filepaths_sim
dfsim.columns=['gal','rc','ID_t6','file'] # columns of dataframe are: x-axis rotation value, y-axis rotation value, #of detector after lens, ID# of file

dfsim['rc']=pd.to_numeric(dfsim['rc'],downcast="float") 
dfsim['gal']=pd.to_numeric(dfsim['gal'],downcast="float")
dfsim['ID_t6']=pd.to_numeric(dfsim['ID_t6'],downcast="integer")
dfsim['file'] = dfsim['file'].astype('str') 

dfsim.dtypes

In [ ]:
# shuffle and check the dataframe

dfsim = dfsim.sample(frac=1).reset_index(drop=True)
dfsim 

#### split dataframe to two: one will serve as test dataset

Split dataframe to a train (75% of initial df length), validation (15% of inital train length), and test (10% of initial df length) dataframes.

In [ ]:
tr=int(len(dfsim.index)*0.75)
v=int(len(dfsim.index)*0.9)
print(tr,v)

dftrain=dfsim.iloc[:tr,:]
dfvalid=dfsim.iloc[tr:v,:]
dftest=dfsim.iloc[v:,:]

#### inspect dataframes

In [ ]:
print('train: ',len(dftrain), '   valid: ',len(dfvalid),'    test dataframe:',len(dftest))

In [ ]:
plt.hist(dfsim['rc'],bins=20)
plt.show()

### Set parameters dictionary

**Edit the following dictionary accordingly. Its entries - some of which are hyperparameters of the model - are used further down and are also saved in Neptune.ai**

In [ ]:
PARAMS = {'lr': 5e-5,   
          'dropout': 0.0,
          'batch_size': 32,
          'n_epochs': 200,
          'optimizer': 'ADAM',
          'loss': 'MAE',
          'metrics': 'RMSE, MAE',
          'activations': 'relu, linear',
          'notebook':'the name of the notebook used',
          'image_input_shape' : (256,256,1),
          'data_description' : ' ',
          'dataset size' : '  ',
          'testset predictions csv' :'testset_name.csv' ,
          'run_name' : 'neptune run ID',
          'save_model' : 'name_of_model_to_save.h5',
          'run env' : ' ',   
           'NOTES': ' General notes'      
          }

### DataGenerators: use flow_from_dataframe class

__create datasets__

In [ ]:
batch = PARAMS['batch_size']

train_datagen = ImageDataGenerator(samplewise_center=True,samplewise_std_normalization=True)
train = train_datagen.flow_from_dataframe(dftrain,x_col='file',y_col=['gal','rc'],class_mode='multi_output',batch_size=batch,target_size=(256,256),color_mode='grayscale')

valid_datagen = ImageDataGenerator(samplewise_center=True,samplewise_std_normalization=True)
validation = valid_datagen.flow_from_dataframe(dfvalid,x_col='file',y_col=['gal','rc'],class_mode='multi_output',batch_size=batch,target_size=(256,256),color_mode='grayscale')

# notice that for the testset generator shuffle is set to False. This way it's possible to copy the real values from the dataset in order to compare with final predictions on the dataset
test_datagen = ImageDataGenerator(samplewise_center=True,samplewise_std_normalization=True)
test = test_datagen.flow_from_dataframe(dftest,x_col='file',y_col=['gal','rc'],class_mode='multi_output',batch_size=batch,target_size=(256,256),color_mode='grayscale',shuffle=False)


### CNN Model

In [ ]:
# Clean session and model if needed

backend.clear_session()

In [ ]:
# Keras does not have RMSE as loss function. Needs to be defined (note: it has MSE though).

from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [ ]:
# The model architecture and a printed summary at the end

input_layer=Input(shape=(256,256,1))

NNlayer=Conv2D(32, (3, 3), activation='relu')(input_layer)
NNlayer=Conv2D(64, (3, 3), activation='relu')(NNlayer)
#NNlayer=BatchNormalization()(NNlayer)      
NNlayer=MaxPool2D((2, 2))(NNlayer)
NNlayer=Conv2D(128, (3, 3), activation='relu')(NNlayer)
#NNlayer=BatchNormalization()(NNlayer)
NNlayer=MaxPool2D((2, 2))(NNlayer)
NNlayer=Conv2D(256, (3, 3), activation='relu')(NNlayer)
#NNlayer=BatchNormalization()(NNlayer)
NNlayer=MaxPool2D((2, 2))(NNlayer)
NNlayer=Conv2D(256, (3, 3), activation='relu')(NNlayer)
#NNlayer=BatchNormalization()(NNlayer)
NNlayer=MaxPool2D((2, 2))(NNlayer)
NNlayer=Flatten()(NNlayer)
NNlayer=Dense(units=128,activation='relu')(NNlayer)
NNlayer=Dropout(PARAMS['dropout'])(NNlayer)
NNlayer= Dense(units=64, activation='relu')(NNlayer)
NNlayer= Dense(units=16, activation='relu')(NNlayer)

# Branching out begins here
gal = Dense(units=1,activation='linear',name='gal')(NNlayer)
rc = Dense(units=1,activation='linear',name='rc')(NNlayer)

model = Model(inputs=input_layer, outputs=[gal,rc])

model.compile(optimizer=optimizers.Adam(learning_rate=PARAMS["lr"]), 
              loss={'gal':'mae','rc':'mae'},
              metrics={"gal":[metrics.RootMeanSquaredError()],"rc":[metrics.RootMeanSquaredError()]})  



model.summary()

In [ ]:
# Create a graph of the model layers

plot_model(model, to_file='model.png',show_shapes=True,rankdir='TB',show_layer_names=True)

### Define callbacks

-----**invoke Neptune and define callbacks**-----

In [ ]:
# Your neptube token shoud go here, within the quotes Ommit if you do not wish to use neptune

%env NEPTUNE_API_TOKEN=" "

In [ ]:
import neptune.new as neptune

run = neptune.init(project='name-your-project')

In [ ]:
run['Parameters']= PARAMS
run['Name'] = PARAMS['run_name']

In [ ]:
from neptune.new.integrations.tensorflow_keras import NeptuneCallback
neptune_monitor = NeptuneCallback(run=run, base_namespace='metrics')

In [ ]:
class TimingCallback(Callback):
  def __init__(self):
    self.logs=[]
  def on_epoch_begin(self,epoch, logs={}):
    self.starttime=time()
  def on_epoch_end(self,epoch, logs={}):
    self.logs.append(time()-self.starttime)

In [ ]:
timing_callback = TimingCallback()
checkpoint=ModelCheckpoint("./model_checkpoint", monitor='val_loss')
callbacks = [neptune_monitor,checkpoint,timing_callback]

### Model fit

-----**train the model**-----

In [ ]:

epoch_num=PARAMS["n_epochs"]

spe=len(train)
val_steps=len(validation)


history = model.fit(train,
                    steps_per_epoch=spe,
                    epochs=epoch_num,
                    validation_data=validation,
                    validation_steps=val_steps,
                    callbacks=callbacks)

model.save(PARAMS['save_model'])
  
run['model/'+PARAMS['save_model']].upload(PARAMS['save_model'])
run['notebook/'+PARAMS['notebook']].upload(PARAMS['notebook'])                                
run['model/model.png'].upload('model.png')                                

metrics = model.metrics_names

test_score = model.evaluate(test)
pred = model.predict(test)



run['test/'+metrics[0]].log(test_score[0])
run['test/'+metrics[1]].log(test_score[1])
run['test/'+metrics[2]].log(test_score[2]) 

                           


dfpred = dftest[['gal','rc']].copy()

dfpred['pred_gal']=pred[0].flatten()
dfpred['pred_rc']=pred[1].flatten()
dfpred.reset_index(drop=True,inplace=True)

dfpred.to_csv(PARAMS['testset predictions csv'], index=False)
run['test/'+PARAMS['testset predictions csv']].upload(PARAMS['testset predictions csv'])



run.stop()

### Plot learning curves

In [ ]:
def learning_curves(model_history):
    fig, axes = plt.subplots(2, 2, figsize=(20, 10))
    axes[0,0].plot(history.history['loss'], label='Total training loss')
    axes[0,0].plot(history.history['val_loss'], label='Total validation loss')
    axes[0,0].set_xlabel('Epochs')
    axes[0,0].legend()
    
    axes[0,1].plot(history.history['rc_loss'], label=' rc training loss')
    axes[0,1].plot(history.history['val_rc_loss'], label='rc validation loss')
    axes[0,1].set_xlabel('Epochs')
    axes[0,1].legend()
    
    axes[1,0].plot(history.history['gal_loss'], label='gal training loss')
    axes[1,0].plot(history.history['val_gal_loss'], label='gal validation loss')
    axes[1,0].set_xlabel('Epochs')
    axes[1,0].legend()

learning_curves(history)

### Evaluation on test dataset

Load saved model if needed.

In [ ]:
model=load_model('your_model.h5')

evaluate on entirety of test dataset 

In [ ]:
test_score = model.evaluate(test)
print('total loss:  ', test_score[0])
print('gal loss: ' , test_score[1])
print('rc loss: ',test_score[2])

### Predict on single images

Load saved model if needed.

In [ ]:
model=load_model('your_model.h5')

In [ ]:
# Randomly choose a file from the test dataframe

index = random.randrange(len(dftest))
img = image.load_img(dftest.iloc[index].file, target_size=(256, 256),color_mode='grayscale')

img_exp = image.img_to_array(img,dtype=None)

mean=img_exp.mean()
std=img_exp.std()
img_exp = (img_exp-mean)/std
img_exp = np.expand_dims(img_exp, axis=0)

plt.figure(figsize = (10,10))
plt.imshow(img_exp[0],cmap='gray')
plt.show()

prediction = model.predict(img_exp)
print('predicted gal =  ',prediction[0][0][0], 'predicted rc =  ',prediction[1][0][0])
print('real gal = ',dftest.iloc[index].gal,'real rc = ',dftest.iloc[index].rc)

### General sandbox area

In [ ]:
print(test_score)

In [ ]:
plt.figure(figsize = (10,10))
plt.imshow(test[0][0][63],cmap='gray')

scatter plots of predicted and real values fpr individual batches of the testset

In [ ]:
pred=model.predict(test)

In [ ]:
batch_num=3

In [ ]:
pred=model.predict(test[batch_num][0])

In [ ]:
from sklearn.metrics import mean_absolute_error
print("y1 MAE:%.4f" % mean_absolute_error(pred[0],test[batch_num][1][0]))
print("y2 MAE:%.4f" % mean_absolute_error(pred[1],test[batch_num][1][1]))

In [ ]:
x_ax = range(80)
plt.figure(figsize=(10,10))
plt.scatter(x_ax,test[batch_num][1][0],  s=10, label="gal-real",c='k')
plt.plot(x_ax,pred[0], label="gal-pred")
plt.legend()
plt.show()

plt.figure(figsize=(10,10))
plt.scatter(x_ax, test[batch_num][1][1],  s=10, label="rc-real",c='r')
plt.plot(x_ax, pred[1], label="rc-pred",c='g')
plt.legend()
plt.show()